In [3]:
from gurobipy import *
import pandas as pd
import numpy

# Model name
m = Model("schedule") 
# No. of teams
T = 31
maxmatches = 82
#Set of Teams
TEAMS = set([i for i in range(1,32)])
DAYS = set([i for i in range(1,173)])
PLAY = set(['HOME','AWAY','REST'])
# Pairs of all teams, u!= v because a team cannot play itself
PAIRS = set([(a,b) for a in TEAMS for b in TEAMS if a != b])
# The objective is to maximize rest for each teams
r = m.addVar(obj=1,vtype=GRB.INTEGER,name='r')
TDP = set([(t,d,z) for t in TEAMS for d in DAYS for z in PLAY])
#x[t,d,z] = 1 if a team t is having p play on d day
x = m.addVars(TDP,vtype=GRB.INTEGER,name='x')
PVD = set([(a,b,z,d) for (a,b) in PAIRS for d in DAYS for z in PLAY if z !='REST'])
#p[a,b,z,d] = 1 if teams a and b are playing at d day
p = m.addVars(PVD,vtype=GRB.INTEGER,name='p')
#seprate set of counting numbeer of matches a team plays
XYZ = set([(t,d,z) for t in TEAMS for d in DAYS for z in PLAY if z != 'REST'])
#c[t,d,z] = 1 if a team is playing on any given day
c = m.addVars(XYZ,vtype=GRB.INTEGER,name='c')
#Each team cannot play more than 1 match on any given day 
#Here we have equated to one because in Venue we have kept home, away and rest so either the team has home away 
#or rest on any particular day
m.addConstrs((quicksum(x[t,d,z] for z in PLAY) ==1 for t in TEAMS for d in DAYS), 'oneMatch')
#Except for rest each home and away game has to have 2 teams
m.addConstrs((quicksum(x[t,d,z] for t in TEAMS) ==2 for d in DAYS for z in PLAY if z != 'REST'),'REST2')
#More than 1 team can have rest on a given day 
m.addConstrs((quicksum(x[t,d,'REST'] for t in TEAMS) >=1 for d in DAYS),'REST1')
#Maximum matches a team can play in a season
m.addConstrs(((c.sum(t,'*') == maxmatches for t in TEAMS)), name='Maxmatches')
#Each team has to play 41 away games and 41 home games
m.addConstrs((quicksum(x[t,d,p] for d in DAYS) ==41 for t in TEAMS for p in PLAY if p !='REST' and 'AWAY'),'41HOMEGames')
m.addConstrs((quicksum(x[t,d,p] for d in DAYS) ==41 for t in TEAMS for p in PLAY if p !='REST' and 'HOME'),'41AWAYGames')
#Maximize number of rest days for any given team 
m.addConstrs((quicksum(x[t,d,'REST'] for d in DAYS) >=r for t in TEAMS),'objectiveConstraint')
#Pair p[a,b,z,d] is 1 if a and b are playing together on d day 
m.addConstrs((p[a,b,z,d] <= x[b,d,z] for z in PLAY if z != 'REST' for d in DAYS for (a,b) in PAIRS), 'TeamB')
m.addConstrs((p[a,b,z,d] >= x[a,d,z] + x[b,d,z] - 1 for z in PLAY if z!='REST' for d in DAYS for (a,b) in PAIRS),'TeamC')
#Identical Games every 14 days
m.addConstrs((quicksum(p[a,b,z,d] for z in PLAY if z != 'REST' for d in DAYS if d == 14 * d) ==1 for (a,b) in PAIRS),'IdenticalGame')
#No team can play 3 consecutive games
m.addConstrs((quicksum(x[t,d,z] for z in PLAY if z != 'REST' for d in DAYS if d == d+2) ==0 for t in TEAMS),'NoConsecutiveGames')
#No 7 consecutive away games
m.addConstr((quicksum(x[t,d,'AWAY'] for t in TEAMS for d in DAYS if d == d+6) ==0),'No7ConsecutiveAwayGames')
#not more than 3 games in 5 days
m.addConstr((quicksum(x[t,d,z] for z in PLAY if z != 'REST' for d in DAYS if d == d+4) ==0),'3Gamesin5Days')
m.write('schedule.lp')
file = open('schedule.lp','r')
print(file.read())
file.close()
m.optimize()
x_m = dict(m.getAttr('X',x))
p_m = dict(m.getAttr('X',p))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Optimize a model with 646837 rows, 346581 columns and 1668947 nonzeros
Model fingerprint: 0xe6c9ee48
Variable types: 0 continuous, 346581 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+01]
Presolve time: 0.87s

Explored 0 nodes (0 simplex iterations) in 1.29 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


GurobiError: Unable to retrieve attribute 'X'

In [4]:
p_m

{(1, 2, 'HOME', 8): 0.0,
 (6, 1, 'HOME', 24): 0.0,
 (4, 2, 'AWAY', 37): 0.0,
 (2, 7, 'AWAY', 22): 0.0,
 (7, 1, 'HOME', 20): 0.0,
 (3, 2, 'HOME', 33): 0.0,
 (6, 4, 'AWAY', 43): 0.0,
 (2, 6, 'AWAY', 45): 0.0,
 (2, 4, 'HOME', 30): 0.0,
 (3, 4, 'HOME', 16): 1.0,
 (3, 1, 'HOME', 15): 0.0,
 (2, 6, 'HOME', 28): 0.0,
 (4, 7, 'AWAY', 13): 0.0,
 (4, 6, 'AWAY', 9): 0.0,
 (4, 5, 'HOME', 20): 0.0,
 (3, 7, 'AWAY', 39): 0.0,
 (6, 3, 'HOME', 35): 0.0,
 (5, 2, 'HOME', 7): 0.0,
 (6, 1, 'AWAY', 3): 0.0,
 (7, 6, 'AWAY', 13): 0.0,
 (6, 7, 'AWAY', 33): 0.0,
 (1, 4, 'AWAY', 34): 0.0,
 (4, 2, 'HOME', 30): 0.0,
 (1, 2, 'HOME', 48): 0.0,
 (1, 7, 'AWAY', 39): 1.0,
 (1, 5, 'HOME', 10): 0.0,
 (3, 5, 'AWAY', 42): 0.0,
 (3, 2, 'HOME', 9): 0.0,
 (5, 1, 'AWAY', 27): 0.0,
 (2, 6, 'HOME', 4): 0.0,
 (4, 7, 'AWAY', 21): 0.0,
 (3, 7, 'HOME', 44): 0.0,
 (5, 4, 'HOME', 36): 0.0,
 (4, 5, 'HOME', 44): 0.0,
 (5, 2, 'AWAY', 2): 0.0,
 (6, 3, 'HOME', 11): 0.0,
 (5, 2, 'HOME', 47): 0.0,
 (5, 7, 'HOME', 20): 0.0,
 (2, 1, 'HOME', 33)

In [4]:
p_m

{(4, 1, 'HOME', 30): 0.0,
 (2, 6, 'HOME', 14): 0.0,
 (1, 7, 'AWAY', 16): 0.0,
 (6, 2, 'HOME', 12): 0.0,
 (2, 3, 'HOME', 16): 0.0,
 (6, 3, 'HOME', 5): 1.0,
 (1, 4, 'HOME', 45): 0.0,
 (5, 7, 'HOME', 14): 0.0,
 (1, 6, 'HOME', 42): 0.0,
 (7, 4, 'AWAY', 21): 0.0,
 (3, 6, 'AWAY', 46): 0.0,
 (7, 2, 'HOME', 21): 0.0,
 (3, 2, 'HOME', 8): 0.0,
 (2, 5, 'HOME', 27): 0.0,
 (2, 4, 'HOME', 31): 0.0,
 (6, 4, 'AWAY', 21): 0.0,
 (7, 3, 'HOME', 38): 0.0,
 (6, 5, 'AWAY', 48): 0.0,
 (5, 6, 'AWAY', 45): 0.0,
 (5, 1, 'HOME', 20): 0.0,
 (5, 3, 'HOME', 21): 0.0,
 (7, 5, 'HOME', 26): 0.0,
 (2, 6, 'HOME', 22): 0.0,
 (4, 6, 'AWAY', 21): 0.0,
 (1, 7, 'AWAY', 8): 0.0,
 (3, 7, 'AWAY', 23): 0.0,
 (1, 3, 'AWAY', 24): 0.0,
 (6, 5, 'HOME', 43): 0.0,
 (6, 7, 'HOME', 6): 0.0,
 (7, 1, 'AWAY', 48): 0.0,
 (2, 7, 'HOME', 24): 0.0,
 (3, 6, 'AWAY', 6): 0.0,
 (3, 2, 'HOME', 32): 0.0,
 (2, 5, 'HOME', 3): 0.0,
 (1, 4, 'AWAY', 40): 0.0,
 (5, 4, 'HOME', 19): 0.0,
 (5, 6, 'AWAY', 5): 0.0,
 (2, 1, 'HOME', 26): 0.0,
 (6, 1, 'HOME', 31)